In [ ]:
from funcs import *
from tqdm import tqdm
import matplotlib

In [ ]:
world_pop = gpd.read_file('./data/pop.geo.json')
world_pop['POP2005']=world_pop['POP2005'].astype(float)
world_pop['area']=  world_pop.to_crs(6933).area.astype(float)*0.000001
world_pop['density'] = (world_pop['POP2005'].div(world_pop['area']))

In [ ]:
country_name_index = {}
for i in range(len(world_pop)):
    row_i = world_pop.iloc[i]
    country_name_index.update({row_i['NAME']:i})

In [ ]:
polygons_ = get_polygons(world_pop['geometry'])
source_polygons = get_source_polygons(world_pop['geometry'])
N_vec_ = world_pop['POP2005'].values
Names_ = world_pop['NAME'].values
Areas_ = world_pop['area'].values

In [ ]:
hx = 0.1
hy = 0.1

In [ ]:
bounds_ = bounds_by_polygons(polygons_)
xgrid,ygrid = make_grids(hx,hy,bounds_)

In [ ]:
fig,ax = plt.subplots()
fig.set_size_inches(16,9)
for i in range(len(polygons_ )):
    ax.plot(polygons_[i][0],polygons_[i][1],c='k')
# for i in range(len(xgrid)):
#     x_ = xgrid[i]
#     ax.plot([x_,x_],[bounds_[1][0],bounds_[1][1]],c='k',linewidth=0.1)
# for i in range(len(ygrid)):
#     y_ = ygrid[i]
#     ax.plot([bounds_[0][0],bounds_[0][1]],[y_,y_],c='k',linewidth=0.1)
# plt.show()

In [ ]:
masks_per_state = get_binary_mask_per_state_cv2(source_polygons,
                                xgrid, ygrid)
mask_of_earth = CUPmasks(masks_per_state)
mask_of_water = np.logical_not(mask_of_earth)

In [ ]:
masks_with_N = [masks_per_state[i].astype(np.float32)*N_vec_[i] for i in range(len(masks_per_state))]

In [ ]:
mask_of_earth_N = None 
for i in range(len(masks_with_N)):
    if mask_of_earth_N is None:
        mask_of_earth_N = masks_with_N[i]
    else:
        mask_of_earth_N += masks_with_N[i]

In [ ]:
cities = pd.read_csv('./data/worldcities.csv')
cities

In [ ]:
cities_mask = np.zeros(shape=(len(xgrid),len(ygrid)),dtype=np.float32)
for i in range(len(cities)):
    x = cities.iloc[i]['lng']
    y = cities.iloc[i]['lat']
    z = cities.iloc[i]['population']
    xi = int((x-xgrid[0])/(xgrid[-1]-xgrid[0])*(len(xgrid)-1))
    yi = int((y-ygrid[0])/(ygrid[-1]-ygrid[0])*(len(ygrid)-1))
    if np.isnan(z):
        z = 10**0
    cities_mask[xi][yi] =  z

In [ ]:
fig,ax = plt.subplots()
fig.set_size_inches(16,9)
cax = ax.matshow(cities_mask.T,cmap='viridis',norm=matplotlib.colors.LogNorm())
ax.invert_yaxis()
fig.colorbar(cax)
ax.set_title('cities')

In [ ]:
fig,ax = plt.subplots()
fig.set_size_inches(16,9)
cax = ax.matshow(np.asarray(mask_of_earth).T,cmap='binary')
ax.invert_yaxis()
fig.colorbar(cax)
ax.set_title('land')

In [ ]:
fig,ax = plt.subplots()
fig.set_size_inches(16,9)
cax = ax.matshow(np.asarray(mask_of_water).T,cmap='binary')
ax.invert_yaxis()
fig.colorbar(cax)
ax.set_title('water')

In [ ]:
fig,ax = plt.subplots()
fig.set_size_inches(16,9)
cax = ax.matshow(np.asarray(mask_of_earth_N).T,cmap='viridis',norm=matplotlib.colors.LogNorm())
ax.invert_yaxis()
fig.colorbar(cax)
ax.set_title(r'$2005г.\ \sum N_i = {}, млрд$'.format(np.sum(N_vec_)/10**9))

In [ ]:
fig,ax = plt.subplots()
fig.set_size_inches(16,9)
cax = ax.matshow(np.asarray(masks_per_state[country_name_index['Russia']]).T,cmap='binary')
ax.invert_yaxis()
fig.colorbar(cax)

In [ ]:
fig,ax = plt.subplots()
fig.set_size_inches(16,9)
cax = ax.matshow(np.asarray(masks_per_state[country_name_index['United States']]).T,cmap='binary')
ax.invert_yaxis()
fig.colorbar(cax)

In [ ]:
poly_per_state = get_polygons_per_state(world_pop['geometry'])

In [ ]:
u = np.zeros(shape=(len(xgrid),len(ygrid)))

In [ ]:
c_mask = np.copy(masks_per_state[country_name_index['Russia']])
c_N = N_vec_[country_name_index['Russia']]
c_grid_count = np.sum(c_mask)
c_boundary_conditions = np.logical_not(c_mask).astype(np.float32)
c_polygons = poly_per_state[country_name_index['Russia']]
c_bounds = [bounds_by_polygons(el) for el in c_polygons]
c_cities_vs = cities_mask*c_mask.astype(np.float32)
c_cities_vs_sum = np.sum(c_cities_vs)
c_cities_vs = (np.sign(c_N-c_cities_vs_sum)*(c_N-c_cities_vs_sum)/np.sum(c_cities_vs>0) + c_cities_vs)*(c_cities_vs>0).astype(np.float32)

In [ ]:
fig,ax = plt.subplots()
fig.set_size_inches(16,9)
cax = ax.matshow(c_cities_vs.T,norm= matplotlib.colors.LogNorm())
ax.invert_yaxis()
fig.colorbar(cax)
ax.set_title(r'$НУ, \Sigma = {}$'.format(np.sum(c_cities_vs)))

In [ ]:
fig,ax = plt.subplots()
fig.set_size_inches(16,9)
cax = ax.matshow(c_boundary_conditions.T,cmap='binary')
ax.invert_yaxis()
fig.colorbar(cax)
ax.set_title('ГУ')